### Carregamento

In [3]:
# Importações necessárias
import pandas as pd

# Função para carregar os dados
def carregar_dados(caminho_arquivo, coluna_data=None):
    """
    Carrega um arquivo CSV com dados.
    
    Parâmetros:
    - caminho_arquivo (str): Caminho do arquivo CSV.
    - coluna_data (str, opcional): Nome da coluna de datas para converter para datetime.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os dados carregados.
    """
    # Carrega o CSV
    dados = pd.read_csv(caminho_arquivo)
    
    # Converte a coluna de datas para datetime com timezone, se especificada
    if coluna_data:
        dados[coluna_data] = pd.to_datetime(dados[coluna_data], utc=True)
    
    return dados

# Exemplo de uso da função com coluna de datas 'date'
caminho_arquivo = '../data/raw/ifood_df.csv'
dados = carregar_dados(caminho_arquivo)

# Exibir as primeiras linhas do DataFrame
dados.head(10)

,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,...,marital_Together,marital_Widow,education_2n Cycle,education_Basic,education_Graduation,education_Master,education_PhD,MntTotal,MntRegularProds,AcceptedCmpOverall
0,58138.0,0,0,58,635,88,546,172,88,88,...,0,0,0,0,1,0,0,1529,1441,0
1,46344.0,1,1,38,11,1,6,2,1,6,...,0,0,0,0,1,0,0,21,15,0
2,71613.0,0,0,26,426,49,127,111,21,42,...,1,0,0,0,1,0,0,734,692,0
3,26646.0,1,0,26,11,4,20,10,3,5,...,1,0,0,0,1,0,0,48,43,0
4,58293.0,1,0,94,173,43,118,46,27,15,...,0,0,0,0,0,0,1,407,392,0
5,62513.0,0,1,16,520,42,98,0,42,14,...,1,0,0,0,0,1,0,702,688,0
6,55635.0,0,1,34,235,65,164,50,49,27,...,0,0,0,0,1,0,0,563,536,0
7,33454.0,1,0,32,76,10,56,3,1,23,...,0,0,0,0,0,0,1,146,123,0
8,30351.0,1,0,19,14,0,24,3,3,2,...,1,0,0,0,0,0,1,44,42,0
9,5648.0,1,1,68,28,0,6,1,1,13,...,1,0,0,0,0,0,1,36,23,1


### Tamanho base

In [4]:
def verificar_tamanho_base(df):
    """
    Verifica o tamanho da base de dados em termos de número de linhas e colunas.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - dict: Dicionário com o número de linhas e colunas.
    """
    tamanho = {
        'Número de Linhas': df.shape[0],
        'Número de Colunas': df.shape[1]
    }
    
    return tamanho

# Verificar o tamanho da base de dados
tamanho_base = verificar_tamanho_base(dados)
tamanho_base

{'Número de Linhas': 2205, 'Número de Colunas': 39}

### Valores nulos

In [5]:
# Função para verificar valores ausentes
def verificar_valores_ausentes(df):
    """
    Verifica a quantidade e o percentual de valores ausentes em cada coluna de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com a quantidade e o percentual de valores ausentes por coluna.
    """
    # Calcular a quantidade de valores ausentes
    valores_ausentes = df.isnull().sum()
    
    # Calcular o percentual de valores ausentes
    percentual_ausentes = (valores_ausentes / df.shape[0]) * 100
    
    # Criar um DataFrame com os resultados
    tabela_ausentes = pd.DataFrame({
        'Quantidade de Valores Ausentes': valores_ausentes,
        'Percentual de Valores Ausentes (%)': percentual_ausentes
    })
    
    # Ordenar o DataFrame pelo percentual de valores ausentes em ordem decrescente
    tabela_ausentes = tabela_ausentes[tabela_ausentes['Quantidade de Valores Ausentes'] > 0]
    tabela_ausentes = tabela_ausentes.sort_values(by='Percentual de Valores Ausentes (%)', ascending=False)
    
    return tabela_ausentes

# Verificar valores ausentes
tabela_ausentes = verificar_valores_ausentes(dados)
tabela_ausentes

,Quantidade de Valores Ausentes,Percentual de Valores Ausentes (%)


### Valores duplicados

In [7]:
def detectar_valores_duplicados(df):
    """
    Detecta e exibe valores duplicados de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado
    
    Retorna:
    - pandas.DataFrame: DataFrame com informações sobre os valores duplicados
    """
    # Verifica duplicatas
    duplicatas = df[df.duplicated(keep=False)]
    
    if duplicatas.empty:
        print("✅ Nenhum valor duplicado foi encontrado.")
        return pd.DataFrame()
    else:
        print(f"⚠️ Foram encontradas {duplicatas.shape[0]} linhas duplicadas.")
        
        # Contar duplicatas por coluna
        duplicatas_contagem = duplicatas.apply(lambda x: x.duplicated(keep=False)).sum()
        
        # Criar DataFrame com informações sobre duplicatas
        tabela_duplicatas = pd.DataFrame({
            'Coluna': duplicatas_contagem.index,
            'Quantidade de Duplicatas': duplicatas_contagem.values
        })
        
        # Filtrar apenas colunas com duplicatas
        tabela_duplicatas = tabela_duplicatas[tabela_duplicatas['Quantidade de Duplicatas'] > 0]
        
        return tabela_duplicatas

# Aplicar a função e exibir a tabela de duplicatas
tabela_duplicatas = detectar_valores_duplicados(dados)
tabela_duplicatas

✅ Nenhum valor duplicado foi encontrado.


""


### Verificação tipos dos dados

In [8]:
# Função para verificar o tipo dos dados/colunas
def verificar_tipo_dados(df):
    """
    Verifica o tipo dos dados/colunas de um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os nomes das colunas e seus respectivos tipos de dados.
    """
    # Obter os tipos de dados das colunas
    tipos_dados = df.dtypes
    
    # Criar um DataFrame com os resultados
    tabela_tipos = pd.DataFrame({
        'Coluna': tipos_dados.index,
        'Tipo de Dado': tipos_dados.values
    })
    
    return tabela_tipos

# Verificar o tipo dos dados/colunas
tabela_tipos = verificar_tipo_dados(dados)
tabela_tipos

,Coluna,Tipo de Dado
0,Income,float64
1,Kidhome,int64
2,Teenhome,int64
3,Recency,int64
4,MntWines,int64
5,MntFruits,int64
6,MntMeatProducts,int64
7,MntFishProducts,int64
8,MntSweetProducts,int64
9,MntGoldProds,int64


### Valores máximos/mínimos

In [15]:
def valores_max_min(df):
    """
    Percorre todas as colunas numéricas (int ou float) de um DataFrame e retorna uma tabela com os valores máximos e mínimos de cada coluna.

    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser analisado.

    Retorna:
    - pandas.DataFrame: DataFrame com os valores máximos e mínimos de cada coluna numérica.
    """
    # Selecionar apenas colunas numéricas
    colunas_numericas = df.select_dtypes(include=['int64', 'float64']).columns
    
    # Criar listas para armazenar os resultados
    colunas = []
    valores_min = []
    valores_max = []
    
    # Percorrer cada coluna numérica e calcular os valores máximo e mínimo
    for coluna in colunas_numericas:
        colunas.append(coluna)
        valores_min.append(df[coluna].min())
        valores_max.append(df[coluna].max())
    
    # Criar o DataFrame final com todos os resultados de uma vez
    tabela_max_min = pd.DataFrame({
        'Coluna': colunas,
        'Valor Mínimo': valores_min,
        'Valor Máximo': valores_max
    })
    
    return tabela_max_min

# Aplicar a função e exibir a tabela de valores máximos e mínimos
tabela_max_min = valores_max_min(dados)
tabela_max_min

,Coluna,Valor Mínimo,Valor Máximo
0,Income,1730.0,113734.0
1,Kidhome,0.0,2.0
2,Teenhome,0.0,2.0
3,Recency,0.0,99.0
4,MntWines,0.0,1493.0
5,MntFruits,0.0,199.0
6,MntMeatProducts,0.0,1725.0
7,MntFishProducts,0.0,259.0
8,MntSweetProducts,0.0,262.0
9,MntGoldProds,0.0,321.0


### Otimização

In [17]:
def modificar_tipo_colunas(df, colunas_tipos):
    """
    Modifica o tipo de dado de colunas específicas em um DataFrame.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame original.
    - colunas_tipos (dict): Dicionário onde as chaves são os nomes das colunas e os valores são os tipos de dados desejados.
    
    Retorna:
    - pandas.DataFrame: DataFrame com os tipos de dados modificados.
    """
    # Verifica se as colunas informadas existem no DataFrame
    colunas_invalidas = [col for col in colunas_tipos.keys() if col not in df.columns]
    if colunas_invalidas:
        raise ValueError(f"Colunas inválidas: {colunas_invalidas}")
    
    # Modifica o tipo de dado das colunas especificadas
    for coluna, tipo in colunas_tipos.items():
        try:
            df[coluna] = df[coluna].astype(tipo)
            print(f"✅ Coluna '{coluna}' convertida para '{tipo}' com sucesso.")
        except Exception as e:
            print(f"⚠️ Erro ao converter a coluna '{coluna}' para '{tipo}': {e}")
    
    return df

# Exemplo de uso:
colunas_tipos = {
    'Kidhome': 'int8',
    'Teenhome': 'int8',
    'Recency': 'int8',
    'NumDealsPurchases': 'int8',
    'NumWebPurchases': 'int8',
    'NumCatalogPurchases': 'int8',
    'NumStorePurchases': 'int8',
    'NumWebVisitsMonth': 'int8',
    'AcceptedCmp3': 'int8',
    'AcceptedCmp4': 'int8',
    'AcceptedCmp5': 'int8',
    'AcceptedCmp1': 'int8',
    'AcceptedCmp2': 'int8',
    'Complain': 'int8',
    'Response': 'int8',
    'marital_Divorced': 'int8',
    'marital_Married': 'int8',
    'marital_Single': 'int8',
    'marital_Together': 'int8',
    'marital_Widow': 'int8',
    'education_2n Cycle': 'int8',
    'education_Basic': 'int8',
    'education_Graduation': 'int8',
    'education_Master': 'int8',
    'education_PhD': 'int8',
    'Z_CostContact': 'int8',
    'Z_Revenue': 'int8',
    'AcceptedCmpOverall': 'int8',
    'Customer_Days': 'int32',  # Ou 'int16'
    'Age': 'int16',  # Ou 'int8'
}
    

dados = modificar_tipo_colunas(dados, colunas_tipos)
dados.dtypes # Verificar os tipos de dados


✅ Coluna 'Kidhome' convertida para 'int8' com sucesso.
✅ Coluna 'Teenhome' convertida para 'int8' com sucesso.
✅ Coluna 'Recency' convertida para 'int8' com sucesso.
✅ Coluna 'NumDealsPurchases' convertida para 'int8' com sucesso.
✅ Coluna 'NumWebPurchases' convertida para 'int8' com sucesso.
✅ Coluna 'NumCatalogPurchases' convertida para 'int8' com sucesso.
✅ Coluna 'NumStorePurchases' convertida para 'int8' com sucesso.
✅ Coluna 'NumWebVisitsMonth' convertida para 'int8' com sucesso.
✅ Coluna 'AcceptedCmp3' convertida para 'int8' com sucesso.
✅ Coluna 'AcceptedCmp4' convertida para 'int8' com sucesso.
✅ Coluna 'AcceptedCmp5' convertida para 'int8' com sucesso.
✅ Coluna 'AcceptedCmp1' convertida para 'int8' com sucesso.
✅ Coluna 'AcceptedCmp2' convertida para 'int8' com sucesso.
✅ Coluna 'Complain' convertida para 'int8' com sucesso.
✅ Coluna 'Response' convertida para 'int8' com sucesso.
✅ Coluna 'marital_Divorced' convertida para 'int8' com sucesso.
✅ Coluna 'marital_Married' conver

Income                  float64
Kidhome                    int8
Teenhome                   int8
Recency                    int8
MntWines                  int64
MntFruits                 int64
MntMeatProducts           int64
MntFishProducts           int64
MntSweetProducts          int64
MntGoldProds              int64
NumDealsPurchases          int8
NumWebPurchases            int8
NumCatalogPurchases        int8
NumStorePurchases          int8
NumWebVisitsMonth          int8
AcceptedCmp3               int8
AcceptedCmp4               int8
AcceptedCmp5               int8
AcceptedCmp1               int8
AcceptedCmp2               int8
Complain                   int8
Z_CostContact              int8
Z_Revenue                  int8
Response                   int8
Age                       int16
Customer_Days             int32
marital_Divorced           int8
marital_Married            int8
marital_Single             int8
marital_Together           int8
marital_Widow              int8
educatio

### Verificação consistência dos dados

In [18]:
def verificar_colunas_numericas(df, colunas):
    """
    Verifica se as colunas especificadas contêm apenas valores numéricos.

    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser verificado.
    - colunas (list): Lista de nomes das colunas a serem verificadas.

    Retorna:
    - dict: Dicionário com o nome da coluna e um booleano indicando se contém apenas números.
    - pandas.DataFrame: DataFrame com as inconsistências encontradas.
    """
    resultado = {}
    inconsistencias = []

    for coluna in colunas:
        if coluna in df.columns:
            # Verifica se todos os valores na coluna são numéricos
            col_numerica = pd.to_numeric(df[coluna], errors='coerce')
            resultado[coluna] = col_numerica.notnull().all()
            if not resultado[coluna]:
                # Adiciona as inconsistências à lista
                posicoes_inconsistentes = col_numerica[col_numerica.isnull()].index.tolist()
                for posicao in posicoes_inconsistentes:
                    inconsistencias.append({'Coluna': coluna, 'Posição': posicao, 'Valor': df.at[posicao, coluna]})
        else:
            resultado[coluna] = False
            print(f"⚠️ Coluna '{coluna}' não encontrada no DataFrame.")

    # Cria um DataFrame com as inconsistências
    df_inconsistencias = pd.DataFrame(inconsistencias)

    return resultado, df_inconsistencias

# Colunas a serem verificadas
colunas_a_verificar = [
    'Income', 'Kidhome', 'Teenhome',
    'Recency', 'MntWines', 'MntFruits', 
    'MntMeatProducts', 'MntFishProducts', 
    'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases',
    'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 
    'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 
    'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2',
    'Complain', 'Z_CostContact', 'Z_Revenue',
    'Response', 'Age', 'Customer_Days', 'marital_Divorced',
    'marital_Married', 'marital_Single', 'marital_Together', 
    'marital_Widow', 'education_2n Cycle', 'education_Basic', 
    'education_Graduation', 'education_Master', 'education_PhD',
    'MntTotal', 'MntRegularProds', 'AcceptedCmpOverall'

]

# Verificar as colunas no DataFrame 'dados'
resultado_verificacao, df_inconsistencias = verificar_colunas_numericas(dados, colunas_a_verificar)

# Exibir o resultado da verificação
for coluna, is_numerica in resultado_verificacao.items():
    status = "contém apenas números" if is_numerica else "não contém apenas números"
    print(f"Coluna '{coluna}': {status}")

# Exibir as inconsistências, se houver
if not df_inconsistencias.empty:
    print("\nInconsistências encontradas:")
    display(df_inconsistencias)
else:
    print("\nNenhuma inconsistência encontrada nas colunas verificadas.")

Coluna 'Income': contém apenas números
Coluna 'Kidhome': contém apenas números
Coluna 'Teenhome': contém apenas números
Coluna 'Recency': contém apenas números
Coluna 'MntWines': contém apenas números
Coluna 'MntFruits': contém apenas números
Coluna 'MntMeatProducts': contém apenas números
Coluna 'MntFishProducts': contém apenas números
Coluna 'MntSweetProducts': contém apenas números
Coluna 'MntGoldProds': contém apenas números
Coluna 'NumDealsPurchases': contém apenas números
Coluna 'NumWebPurchases': contém apenas números
Coluna 'NumCatalogPurchases': contém apenas números
Coluna 'NumStorePurchases': contém apenas números
Coluna 'NumWebVisitsMonth': contém apenas números
Coluna 'AcceptedCmp3': contém apenas números
Coluna 'AcceptedCmp4': contém apenas números
Coluna 'AcceptedCmp5': contém apenas números
Coluna 'AcceptedCmp1': contém apenas números
Coluna 'AcceptedCmp2': contém apenas números
Coluna 'Complain': contém apenas números
Coluna 'Z_CostContact': contém apenas números
Colun

### Carga

In [19]:
def salvar_dados(df, caminho_arquivo_saida):
    """
    Salva o DataFrame em um arquivo CSV.
    
    Parâmetros:
    - df (pandas.DataFrame): DataFrame a ser salvo.
    - caminho_arquivo_saida (str): Caminho do arquivo CSV de saída.
    """
    df.to_csv(caminho_arquivo_saida, index=False)
    print(f"✅ Dados salvos com sucesso em '{caminho_arquivo_saida}'")

# Exemplo de uso:
caminho_arquivo_saida = '../data/processed/ifood_df_atualizado.csv'
salvar_dados(dados, caminho_arquivo_saida)

✅ Dados salvos com sucesso em '../data/processed/ifood_df_atualizado.csv'
